In [14]:
using Gen, PyCall, Plots
import Random, Logging

Logging.disable_logging(Logging.Info);
Random.seed!(42)


Random.TaskLocalRNG()

In [3]:
#dataset
function make_dataset_line(n)
    # true parameters
    prob_outlier = 0.2
    true_inlier_noise = 0.5
    true_outlier_noise = 5.0

    true_slope = randn()*2
    true_intercept = randn()*5
    #print the true parameters
    println("True slope: ", true_slope)
    println("True intercept: ", true_intercept)

    xs = collect(range(-5, stop=5, length=n))
    ys = Float64[]
    for (i, x) in enumerate(xs)
        if randn() < prob_outlier
            y = randn() * true_outlier_noise
        else
            y = true_slope * x + true_intercept + randn() * true_inlier_noise
        end
        push!(ys, y)
    end
    (xs, ys)
end


make_dataset_line (generic function with 1 method)

In [4]:
#model, adapted from playground to only use normal distributions
@gen function regression_line(xs::Vector{<:Real})
    # ...

    slope ~ normal(0, 2)
    intercept ~ normal(2, 2)

    noise ~ normal(0.2, 0.03)
    prob_outlier ~ normal(0, 1)

    # Next, we generate the actual y coordinates.
    n = length(xs)
    ys = Float64[]

    for i = 1:n
        # Decide whether this point is an outlier, and set
        # mean and standard deviation accordingly
        outlier= {:data => (i, :is_outlier)} ~ normal(prob_outlier,0.5)

        if outlier > 0
            (mu, std) = (0., 10.)
        else
            (mu, std) = (xs[i] * slope + intercept, noise)
        end
        # Sample a y value for this point

        y={:data => i => :y} ~ normal(mu, std)

        push!(ys, y)

    end
    ys
end;


In [5]:
@gen function testProposal_line(trace)

    slope ~ normal(trace[:slope], 0.1)
    intercept ~ normal(trace[:intercept], 0.1)
    noise ~ normal(trace[:noise], 0.01)
    prob_outlier ~ normal(trace[:prob_outlier], 0.01)

    for i =1:10
        outlier=@trace(normal(prob_outlier,0.5),(:data => i => :is_outlier))
        if outlier > 0
            (mu, std) = (0., 10.)
        else
            (mu, std) = (xs[i] * slope + intercept, noise)
        end
        # Sample a y value for this point
        y=@trace(normal(mu, std), (:data,i,:y))
    end
    return nothing
end;


In [6]:
#nn model line
#inputs are:                                values
# - x values                                10
# - y values                                10
# - address of sample (one hot encoded)     5
# - instance id (one hot encoded)           10

# - if lstm: previous hidden state

#outputs are:
# - mean
# - std

tf = pyimport("tensorflow")
keras = pyimport("keras")

def nn_line():
    nn_model = keras.models.Sequential()
    nn_model.add(keras.layers.Dense(10, input_shape=(35,), activation="relu"))
    nn_model.add(keras.layers.Dense(10, activation="relu"))
    nn_model.add(keras.layers.Dense(2, activation="linear"))

    #compile the model
    nn_model.compile(loss="mse", optimizer="adam")

    #train the model

    #predict the mean and std
    test=rand(1,35)
    mean, std = nn_model.predict(test)
    return nn_model


PyCall.PyError: PyError (PyImport_ImportModule

The Python package tensorflow could not be imported by pyimport. Usually this means
that you did not install tensorflow in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the tensorflow module, you can
use `pyimport_conda("tensorflow", PKG)`, where PKG is the Anaconda
package that contains the module tensorflow, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ModuleNotFoundError'>
ModuleNot

In [7]:
# main

xs, ys = make_dataset(10)

constraints = choicemap()
for (i, y) in enumerate(ys)
    constraints[:data => i => :y] = y
end

(trace, _) = Gen.generate(regression, (xs,), constraints)

# include a proposal
for i=1:10
    (trace, _) = Gen.mh(trace, testProposal, ())
end

println(trace)


UndefVarError: UndefVarError: `make_dataset` not defined

In [8]:
@gen function coin_model()
    val1 ~ normal(0, 1)
    if val1 > 0
        val2 ~ normal(0.5, 0.5)
    else
        val2 ~ normal(-0.5, 0.5)
    end
end;


In [9]:
@gen function coin_proposal(trace)
    val1 ~ normal(trace[:val1], 0.1)
end;


In [10]:
function block_resimulation_inference_coin(val2)
    observations = Gen.choicemap()
    observations[:val2] = val2
    (tr, ) = Gen.generate(coin_model, (), observations)
    for iter=1:10000
        (tr, ) = mh(tr, coin_proposal, ())
        tr
    end
    tr
end;


In [11]:
trace = Gen.simulate(coin_model, ());
choices=Gen.get_choices(trace)
val2=choices[:val2]
println("True value")
println(choices[:val1])
println(choices[:val2])

tr = block_resimulation_inference(val2)
choices=Gen.get_choices(tr)

println("Inferred value")
println(choices[:val1])
println(choices[:val2])


True value
0.7883556016042917
0.06007070202280035


UndefVarError: UndefVarError: `block_resimulation_inference` not defined

In [15]:
@gen function burglar_model(chance)
    burglar ~ bernoulli(chance)

    if burglar
        alarm ~ bernoulli(0.9)
    else
        alarm ~ bernoulli(0.15)
    end

    if alarm
        john ~ bernoulli(0.6)
        mary ~ bernoulli(0.7)
    else
        john ~ bernoulli(0.3)
        mary ~ bernoulli(0.1)
    end

end;


In [16]:

using PyCall

# Import PyTorch
nn = pyimport("torch.nn")
F = pyimport("torch.nn.functional")
# generate training data by sampling from the model:
function generate_data(model, n, chance)
    data = []
    for i=1:n
        (trace, _) = Gen.generate(model, (chance,), choicemap())
        push!(data, trace)
    end
    return data
end

data = generate_data(burglar_model, 100, .5)
data


100-element Vector{Any}:
 Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##burglar_model#296", Bool[0], false), Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(:burglar => Gen.ChoiceOrCallRecord{Bool}(false, -0.6931471805599453, NaN, true), :alarm => Gen.ChoiceOrCallRecord{Bool}(false, -0.16251892949777494, NaN, true), :mary => Gen.ChoiceOrCallRecord{Bool}(false, -0.10536051565782628, NaN, true), :john => Gen.ChoiceOrCallRecord{Bool}(false, -0.35667494393873245, NaN, true)), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}()), false, -1.3177015696542789, 0.0, (0.5,), false)
 Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##burglar_model#296", Bool[0], false), Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(:bur

In [17]:
using PyCall

# Import PyTorch
nn = pyimport("torch.nn")
F = pyimport("torch.nn.functional")

function y_from_trace(trace)
    # return the value of y (mary, john) from the trace
    choices = Gen.get_choices(trace)
    return choices[:mary], choices[:john]
end

function x_from_trace(trace)
    # return the value of x (burglar, alarm) from the trace
    choices = Gen.get_choices(trace)
    return choices[:burglar], choices[:alarm]
end

@pydef mutable struct NeuralProposal <: nn.Module
    function __init__(self, num_in, num_out)
        # Note the use of pybuiltin(:super): built in Python functions
        # like `super` or `str` or `slice` are all accessed using
        # `pybuiltin`.
        pybuiltin(:super)(NeuralProposal, self).__init__()
        self.fc1 = nn.Linear(num_in, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, num_out)
    end

    function forward(self, x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # make sure the output is between 0 and 1
        x = F.sigmoid(x)
        return x
    end
end


# input is tuple (john, mary)
# output is parameters for (burglar, alarm)
network = NeuralProposal(2, 2)
network


PyObject NeuralProposal(
  (fc1): Linear(in_features=2, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [18]:
@gen function simple_neural_proposal(trace)
    parameters = network(torch.tensor([trace[:john], trace[:mary]], dtype=torch.float32))
    # extract float values from the PyTorch tensor
    b_param = parameters[1].item()
    a_param = parameters[2].item()

    burglar ~ bernoulli(b_param)
    alarm ~ bernoulli(a_param)
end;


function block_resimulation_burglar(john,mary, budget=1000, chance=0.5)
    observations = Gen.choicemap()
    observations[:john] = john
    observations[:mary] = mary
    (tr, ) = Gen.generate(burglar_model, (chance,), observations)
    for iter=1:budget
        (tr, ) = mh(tr, select(:burglar, :alarm))
    end
    tr
end;

function nn_resimulation_burglar(john,mary, budget=1000, chance=0.5)
    observations = Gen.choicemap()
    observations[:john] = john
    observations[:mary] = mary
    (tr, ) = Gen.generate(burglar_model, (chance,), observations)
    for iter=1:budget
        (tr, ) = mh(tr, simple_neural_proposal, ())
    end
    tr
end;


function make_constraints_burglar(trace)
    choices = Gen.get_choices(trace)
    constraints = choicemap()
    constraints[:john] = choices[:john]
    constraints[:mary] = choices[:mary]
    constraints[:burglar] = choices[:burglar]
    constraints[:alarm] = choices[:alarm]
    return constraints
end

trace = Gen.simulate(burglar_model, (0.5,));

println("True value")
c=make_constraints_burglar(trace)
println(c)

tr = block_resimulation_burglar(c[:john],c[:mary])
choices=Gen.get_choices(tr)

println("Inferred value")
c=make_constraints_burglar(tr)
c


True value


DynamicChoiceMap(

Dict{Any, Any}(

:burglar => false, :alarm => false, :mary => false, :john => false), Dict{Any, Any}())


Inferred value


│
├── :burglar : false
│
├── :alarm : false
│
├── :mary : false
│
└── :john : false


In [60]:
# how often does it get it right?
function accuracy_burglar(model, data, inference, budget=1000, chance=0.5)
    correct = 0
    for trace in data
        c = make_constraints_burglar(trace)
        tr = inference(c[:john],c[:mary], budget, chance)
        choices=Gen.get_choices(tr)
        if choices[:burglar] == c[:burglar]
            correct += 1
        end
    end
    return correct / length(data)
end

chance = 0.5
data = generate_data(burglar_model, 1000, chance)

accuracy_burglar(burglar_model, data, block_resimulation_burglar, 100, chance)


0.621

In [71]:
accuracy_burglar(burglar_model, data, nn_resimulation_burglar, 10, chance)


0.599

In [21]:
# now lets train the neural network

# first, we need to convert the data into a format that PyTorch can use
data = generate_data(burglar_model, 1000, chance)

# the loss function is the negative log likelihood
loss_fn = nn.BCELoss()

# the optimizer is the algorithm that updates the parameters
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)

# we will train for 100 epochs
epochs = 100

for epoch=1:epochs
    losses = []
    for trace in data
        # convert the trace into a tensor
        x = torch.tensor([trace[:john], trace[:mary]], dtype=torch.float32)
        y = torch.tensor([trace[:burglar], trace[:alarm]], dtype=torch.float32)

        # compute the output of the network
        y_pred = network(x)

        # compute the loss
        loss = loss_fn(y_pred, y)
        # add loss
        push!(losses, loss.item())

        # zero the gradients before running the backward pass.
        optimizer.zero_grad()

        # run the backward pass
        loss.backward()

        # update the parameters
        optimizer.step()
    end

    mean_loss = sum(losses) / length(losses)
    println(epoch, ": loss: ", mean_loss)
end


1: loss: 0.5626043429896236


2: loss: 0.5435521984696389


3: loss: 0.5432747783586382


4: loss: 0.5427235867902637


5: loss: 0.5430796567872167


...

18: loss: 0.5417709210813045


19: loss: 0.5417040618881583


20: loss: 0.5415794762372971


21: loss: 0.5416261689588427


InterruptException: InterruptException:

In [33]:
using PyCall
torch = pyimport("torch")
DataLoader = torch.utils.data.DataLoader

# convert the data into a format that PyTorch can use
data = generate_data(burglar_model, 100000, chance)

# convert data to tensors and zip them
data_tensors = [(torch.tensor([trace[:john], trace[:mary]], dtype=torch.float32),
                 torch.tensor([trace[:burglar], trace[:alarm]], dtype=torch.float32)) for trace in data]
data_tensors


100000-element Vector{Tuple{PyObject, PyObject}}:
 (PyObject tensor([0., 0.]), PyObject tensor([0., 0.]))
 (PyObject tensor([1., 0.]), PyObject tensor([0., 0.]))
 (PyObject tensor([1., 1.]), PyObject tensor([1., 1.]))
 (PyObject tensor([1., 0.]), PyObject tensor([1., 1.]))
 (PyObject tensor([0., 0.]), PyObject tensor([0., 0.]))
 (PyObject tensor([1., 0.]), PyObject tensor([0., 0.]))
 (PyObject tensor([1., 1.]), PyObject tensor([1., 1.]))
 (PyObject tensor([1., 1.]), PyObject tensor([1., 1.]))
 (PyObject tensor([1., 1.]), PyObject tensor([0., 0.]))
 (PyObject tensor([1., 1.]), PyObject tensor([0., 1.]))
 ⋮
 (PyObject tensor([0., 1.]), PyObject tensor([1., 1.]))
 (PyObject tensor([1., 1.]), PyObject tensor([0., 0.]))
 (PyObject tensor([1., 1.]), PyObject tensor([1., 1.]))
 (PyObject tensor([1., 0.]), PyObject tensor([1., 1.]))
 (PyObject tensor([0., 1.]), PyObject tensor([0., 0.]))
 (PyObject tensor([0., 0.]), PyObject tensor([0., 0.]))
 (PyObject tensor([0., 0.]), PyObject tensor([1., 0

In [70]:
network = NeuralProposal(2, 2)
# create a DataLoader with batch size
batch_size = 1000
data_loader = DataLoader(data_tensors, batch_size=batch_size, shuffle=true)

# the loss function is the negative log likelihood
loss_fn = nn.BCELoss()

# the optimizer is the algorithm that updates the parameters
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)

# we will train for 100 epochs
epochs = 10

for epoch=1:epochs
    losses = []
    for (x, y) in data_loader
        # compute the output of the network
        y_pred = network(x)

        # compute the loss
        loss = loss_fn(y_pred, y)
        # add loss
        push!(losses, loss.item())

        # zero the gradients before running the backward pass.
        optimizer.zero_grad()

        # run the backward pass
        loss.backward()

        # update the parameters
        optimizer.step()
    end

    mean_loss = sum(losses) / length(losses)
    println(epoch, ": loss: ", mean_loss)
end


1: loss: 0.5245602822303772


2: loss: 0.5163607007265091


3: loss: 0.5168133103847503


4: loss: 0.5162248337268829


5: loss: 0.516531158387661


6: loss: 0.5169354456663132


7: loss: 0.5165318793058395


8: loss: 0.5165908205509185


9: loss: 0.5169762372970581


10: loss: 0.5167744207382202


In [23]:
using PyCall

# Import PyTorch
nn = pyimport("torch.nn")
F = pyimport("torch.nn.functional")

@pydef mutable struct LSTMProposal <: nn.Module
    function __init__(self; input_size::Int, hidden_size::Int, num_layers::Int, output_size::Int)
        pybuiltin(:super)(LSTMProposal, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=true)
        self.linear = nn.Linear(hidden_size, output_size)
    end

    function forward(self, x)
        # Assuming x is of shape (batch, seq, feature)
        lstm_out, (h_n, c_n) = self.lstm(x)
        # We are taking the output of the last sequence step
        last_seq_out = lstm_out[:, end, :]
        out = self.linear(last_seq_out)
        return out
    end
end


PyObject <class 'LSTMProposal'>

In [24]:
torch = pyimport("torch")
nn = torch.nn
F = nn.functional

function get_nn():
    # inputs: observe = [john, mary], address (one hot), x_t-1
    # outputs: mean, std
    # LSTM
    # input size: 2 (y: john, mary) + 2 (address: burglar or alarm) + 1 (x_t-1) = 5
    input_size = 5
    hidden_size = 10
    num_layers = 1
    output_size = 2
    model = nn.LSTM(input_size, hidden_size, num_layers)

    return model
end


function train(model, inputs, outputs, epochs)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch=1:epochs
        optimizer.zero_grad()

        outputs_pred = model(inputs)
        loss = criterion(outputs_pred, outputs)

        loss.backward()
        optimizer.step()
    end
end


ErrorException: syntax: newline not allowed after ":" used for quoting

In [25]:
@gen function burglar_proposal(trace)
    burglar ~ bernoulli(0.6)
end;


In [26]:
function block_resimulation_burglar(john,mary)
    observations = Gen.choicemap()
    observations[:john] = john
    observations[:mary] = mary
    (tr, ) = Gen.generate(burglar_model, (0.2,), observations)
    for iter=1:10000
        (tr, ) = mh(tr, burglar_proposal, ())
        (tr, ) = mh(tr, select(:alarm))
    end
    tr
end;


In [27]:
trace = Gen.simulate(burglar_model, (0.6,));

println("True value")
c=make_constraints_burglar(trace)
println(c)

tr = block_resimulation_burglar(c[:john],c[:mary])
choices=Gen.get_choices(tr)

println("Inferred value")
c=make_constraints_burglar(tr)
c


True value
DynamicChoiceMap(Dict{Any, Any}(:burglar => true, :alarm => true, :mary => true, :john => false), Dict{Any, Any}())


Inferred value


│
├── :burglar : false
│
├── :alarm : false
│
├── :mary : true
│
└── :john : false
